# Movie Recommendation Project

In [1]:
#!mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
#!ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

## Data Load

In [41]:
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


In [42]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [43]:
# 3점 미만 데이터는 선호하지 않는다고 가정, 3점 이상만 
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [44]:
# 별점을 시청횟수로 가정, ratings 컬럼의 이름을 counts로 변경
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [45]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [46]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### Setting for Model Validation

In [47]:
# 좋아하는 영화찾기
search_titles = ['Titanic' ,'Meet Joe Black' ,'Chungking Express', 'Notting Hill', 'While You Were Sleeping']
pattern = '|'.join(search_titles)  # 검색할 영화 제목들을 하나의 문자열로 결합

filtered_movies = movies[movies['title'].str.contains(pattern, case=False)]

print(filtered_movies)

      movie_id                                   title                  genre
121        123                Chungking Express (1994)  Drama|Mystery|Romance
335        339          While You Were Sleeping (1995)         Comedy|Romance
1672      1721                          Titanic (1997)          Drama|Romance
2088      2157  Chambermaid on the Titanic, The (1998)                Romance
2271      2340                   Meet Joe Black (1998)                Romance
2602      2671                     Notting Hill (1999)         Comedy|Romance
3334      3403                Raise the Titanic (1980)         Drama|Thriller
3335      3404                          Titanic (1953)           Action|Drama


In [48]:
my_favorite = [123, 1721, 2340, 2671, 339]

my_playlist = pd.DataFrame({'user_id': ['dy']*5, 'movie_id': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':['dy']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = pd.concat([ratings, my_playlist])                            # 위에 임의로 만든 my_playlist 데이터 추가

ratings.tail(10)  

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,dy,123,5,NaN
1,dy,1721,5,NaN
2,dy,2340,5,NaN
3,dy,2671,5,NaN
4,dy,339,5,NaN


In [49]:
# ratings에 movies의 title 가져오기
ratings = pd.merge(ratings, movies[['movie_id', 'title']], on='movie_id', how='left')

ratings.head()

,user_id,movie_id,counts,timestamp,title
0,1,1193,5,978300760.0,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109.0,James and the Giant Peach (1996)
2,1,914,3,978301968.0,My Fair Lady (1964)
3,1,3408,4,978300275.0,Erin Brockovich (2000)
4,1,2355,5,978824291.0,"Bug's Life, A (1998)"


In [51]:
ratings = ratings[['user_id','counts','title']]

## EDA

In [52]:
# 유저 수
ratings['user_id'].nunique()

6040

In [53]:
# 아티스트 수
ratings['title'].nunique()

3628

In [54]:
# 인기 많은 아티스트
title_count = ratings.groupby('title')['user_id'].count()
title_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [55]:
# 유저 별 영화 수 describe
user_count = ratings.groupby('user_id')['title'].count()
user_count.describe()

count    6040.000000
mean      138.490563
std       156.238108
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

In [56]:
# # 유저 별 영화 시청 수 중앙값에 대한 describe
user_median = ratings.groupby('user_id')['counts'].median()
user_median.describe()

count    6040.000000
mean        4.056126
std         0.432278
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

## Data Processing

In [57]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 유저, 아티스트 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [59]:
# 인덱싱 확인
print(user_to_idx['dy'])    
print(movie_to_idx['Titanic (1997)'])

6039
27


In [60]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_title_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_title_data
else:
    print('title column indexing Fail!!')


user_id column indexing OK!!
title column indexing OK!!


/tmp/ipykernel_147/790864692.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다.
/tmp/ipykernel_147/790864692.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['title'] = temp_title_data


In [61]:
# CSR Matrix
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## Matrix Factorization

In [62]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [109]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=500, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)

In [110]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [111]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

In [112]:
dy, chungking_Express = user_to_idx['dy'], movie_to_idx['Chungking Express (1994)']
dy_vector, chungking_Express_vector = als_model.user_factors[dy], als_model.item_factors[chungking_Express]

In [113]:
dy_vector

array([ 4.93188389e-02, -4.79438871e-01, -4.02681917e-01, -1.77008465e-01,
       -2.18683824e-01,  8.11887067e-03, -1.27377674e-01,  8.04519951e-02,
        4.76505786e-01,  3.82747725e-02,  1.48326546e-01, -7.92770181e-03,
       -1.43702105e-01,  7.71987140e-02, -6.04031868e-02,  2.69848943e-01,
       -2.41968594e-03, -3.40500981e-01,  4.78845090e-02,  3.89274687e-01,
       -4.62642163e-02,  2.27210790e-01, -1.71771809e-01,  2.25156605e-01,
       -2.65089065e-01,  7.38661066e-02,  2.61657953e-01,  2.10383371e-01,
       -8.48378241e-02, -2.67342925e-01,  4.01794612e-02, -1.41866654e-01,
        4.16932404e-01,  3.64845842e-01, -9.37797409e-03,  7.21494332e-02,
        1.04372613e-01, -4.16950762e-01,  2.11399689e-01, -9.07784700e-02,
        1.02417901e-01, -2.49361917e-01,  3.98586601e-01, -2.58999884e-01,
        9.59573239e-02, -2.04677105e-01, -4.07164805e-02,  1.78050697e-01,
       -1.63085431e-01,  5.09978607e-02,  5.63800596e-02, -2.45965168e-01,
       -9.50916409e-02, -

In [114]:
chungking_Express_vector

array([ 4.20000078e-03, -4.22790414e-03, -1.09434100e-02,  1.30871618e-02,
        4.71250527e-03,  5.38808154e-03,  9.91730299e-03,  8.49854946e-03,
        9.80592426e-03,  2.13902839e-03,  1.41994227e-02,  2.70013930e-03,
       -3.28044896e-03,  9.47448704e-03, -4.70395200e-03,  1.86641067e-02,
        6.44890266e-03, -3.83072183e-03,  8.36644415e-03,  8.54921620e-03,
        1.05281416e-02,  1.66325942e-02, -1.51297706e-03,  8.91278591e-03,
        1.01194987e-02,  3.65036237e-03,  1.28280348e-03,  3.23747471e-03,
       -8.23793560e-03,  9.37379990e-03,  4.03052988e-03,  5.53669920e-03,
        1.57793127e-02,  1.35972984e-02,  5.30600036e-03,  1.34736560e-02,
        5.14678890e-03,  9.05845314e-03,  8.54836404e-03,  9.51712299e-03,
        8.46140180e-03,  7.20988773e-03,  1.61384605e-02,  7.41991727e-03,
        1.31985908e-02, -1.20539693e-02,  8.55060760e-03,  1.21501097e-02,
        6.01228233e-03,  1.37608964e-02,  1.05439154e-02,  5.26627386e-03,
        4.26602131e-03,  

In [115]:
np.dot(dy_vector, chungking_Express_vector)

0.42293596

In [116]:
Jumanji = movie_to_idx['Jumanji (1995)']
Jumanji_vector = als_model.item_factors[Jumanji]
np.dot(dy_vector, Jumanji_vector)

-0.011131025

In [117]:
Before_Sunrise = movie_to_idx['Before and After (1996)']
Before_Sunrise_vector = als_model.item_factors[Before_Sunrise]
np.dot(dy_vector, Before_Sunrise_vector)

-0.016017193

In [118]:
Notting_Hill = movie_to_idx['Notting Hill (1999)']
Notting_Hill_vector = als_model.item_factors[Notting_Hill]
np.dot(dy_vector, Notting_Hill_vector)

0.8118157

In [119]:
Raise_the_Titanic = movie_to_idx['Raise the Titanic (1980)']
Raise_the_Titanic_vector = als_model.item_factors[Raise_the_Titanic]
np.dot(dy_vector, Raise_the_Titanic_vector)

0.009772317

## Similar Artist Recommendation

In [120]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [121]:
get_similar_movie('Toy Story 2 (1999)')

['Toy Story 2 (1999)',
 'Toy Story (1995)',
 'Tigger Movie, The (2000)',
 'Careful (1992)',
 'Couch in New York, A (1996)',
 'Amityville Curse, The (1990)',
 "Pot O' Gold (1941)",
 'Nosferatu a Venezia (1986)',
 'Digimon: The Movie (2000)',
 'Vie est belle, La (Life is Rosey) (1987)']

In [122]:
get_similar_movie('Chungking Express (1994)')

['Chungking Express (1994)',
 "Ed's Next Move (1996)",
 'Ashes of Time (1994)',
 'Hurricane Streets (1998)',
 'Gate of Heavenly Peace, The (1995)',
 'Aiqing wansui (1994)',
 'Zachariah (1971)',
 'Cement Garden, The (1993)',
 'Master Ninja I (1984)',
 'Mirage (1995)']

In [123]:
get_similar_movie('Titanic (1997)')

['Titanic (1997)',
 'Walking Dead, The (1995)',
 'Paris, France (1993)',
 'Falling in Love Again (1980)',
 'Gay Deceivers, The (1969)',
 'Held Up (2000)',
 'Stranger in the House (1997)',
 'Carnosaur 2 (1995)',
 'Jerry & Tom (1998)',
 'Savior (1998)']

In [124]:
get_similar_movie('Meet Joe Black (1998)')

['Meet Joe Black (1998)',
 "Barney's Great Adventure (1998)",
 'Grandfather, The (El Abuelo) (1998)',
 'Callejón de los milagros, El (1995)',
 'Temptress Moon (Feng Yue) (1996)',
 "Let's Talk About Sex (1998)",
 'Oxygen (1999)',
 'Race the Sun (1996)',
 'Tarzan and the Lost City (1998)',
 'Mirage (1995)']

# Retrospective

처음 validation을 위한 my_playlist에 장르가 무작위인 영화를 설정 후
counts를 5로 설정하였음에도 user_vactor와 movie_vactor의 내적이 0.1점대로 낮은 점수가 나왔었는데

my_playlist에 장르를 통일 후 
Model의 Hyperparameter 중 factors, iterations를 높여 학습하니 my_playlist에 있는 영화는 0.42,0.81로 다른 영화보다 높게 나온 것을 확인할 수 있었다.

similar movie recommendation도 확인하였을 때 장르가 같거나 같은 국적의 영화를 추천해주어
간단하지만 효과적인 것으로 보여진다.